In [14]:
import sys
import numpy
from statsmodels import robust

In [1]:
import findspark

if os.path.isdir("/cluster/home/simondi/spark/"):
    is_cluster = True
    import sparkhpc
    from sparkhpc import sparkjob
    spark_path = "/cluster/home/simondi/spark/"
else:
    is_cluster = False
    spark_path = "/usr/local/spark/spark/"

findspark.init(spark_path)
import pyspark

In [2]:
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [46]:
from pyspark.sql.types import DoubleType

In [3]:
conf = pyspark.SparkConf().setMaster("local").setAppName("test").set("spark.driver.memory", "5G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)

In [4]:
spark = pyspark.sql.SparkSession(sc)

In [5]:
file_name = "/Users/simondi/PHD/data/data/target_infect_x/query_data/sample_10_mock.tsv"

In [6]:
df = spark.read.csv(path=file_name, sep="\t", header='true')
df.cache()

[Row(study='infectx', pathogen='mock', library='d', design='p', replicate='1', plate='zurich-bzok-58', well='a01', gene='atp6v1a', sirna='l-017590-01', well_type='control', image_idx='5', object_idx='162', cells.areashape_area='5067.0', cells.areashape_eccentricity='0.889584183693', cells.areashape_extent='0.647953987122', cells.areashape_formfactor='0.20114801824099998', cells.areashape_majoraxislength='126.163040161', cells.areashape_minoraxislength='57.6276512146', cells.areashape_orientation='-86.2873001099', cells.areashape_perimeter='561.629516602', cells.children_bacteria_count='0', cells.children_invasomes_count='0', cells.location_center_x='1348.18310547', cells.location_center_y='54.2912979126', cells.neighbors_anglebetweenneighbors_2='121.85298156700001', cells.neighbors_firstclosestobjectnumber_2='169.0', cells.neighbors_firstclosestxvector_2='35.7164268494', cells.neighbors_firstclosestyvector_2='26.335483551', cells.neighbors_numberofneighbors_2='4.0', cells.neighbors_per

In [12]:
old_cols = df.schema.names

In [13]:
new_cols = list(map(lambda x: x.replace(".", "_"),  old_cols))

In [20]:
new_cols

['study',
 'pathogen',
 'library',
 'design',
 'replicate',
 'plate',
 'well',
 'gene',
 'sirna',
 'well_type',
 'image_idx',
 'object_idx',
 'cells_areashape_area',
 'cells_areashape_eccentricity',
 'cells_areashape_extent',
 'cells_areashape_formfactor',
 'cells_areashape_majoraxislength',
 'cells_areashape_minoraxislength',
 'cells_areashape_orientation',
 'cells_areashape_perimeter',
 'cells_children_bacteria_count',
 'cells_children_invasomes_count',
 'cells_location_center_x',
 'cells_location_center_y',
 'cells_neighbors_anglebetweenneighbors_2',
 'cells_neighbors_firstclosestobjectnumber_2',
 'cells_neighbors_firstclosestxvector_2',
 'cells_neighbors_firstclosestyvector_2',
 'cells_neighbors_numberofneighbors_2',
 'cells_neighbors_percenttouching_2',
 'cells_neighbors_secondclosestobjectnumber_2',
 'cells_neighbors_secondclosestxvector_2',
 'cells_neighbors_secondclosestyvector_2',
 'cells_parent_nuclei',
 'nuclei_areashape_area',
 'nuclei_areashape_eccentricity',
 'nuclei_area

In [10]:
df = reduce(lambda data, idx: data.withColumnRenamed(old_cols[idx], new_cols[idx]), range(len(new_cols)), df)
for i, x in enumerate(new_cols):
    if x.startswith("cells") or x.startswith("perinuc") or x.startswith("nuclei"):
        df = df.withColumn(x, df[x].cast("double"))

In [44]:
w = Window().partitionBy(["study", "pathogen", "library", "design", "replicate", "plate"]).rowsBetween(-sys.maxsize, sys.maxsize)

In [54]:
from pyspark.sql.types import IntegerType
slen = func.udf(lambda s: len(s), IntegerType())

In [56]:
df.withColumn("k",  slen("sirna").over(w)).show()

AnalysisException: "Expression '<lambda>(sirna#1215)' not supported within a window function.;;\nProject [study#743, pathogen#802, library#861, design#920, replicate#979, plate#1038, well#1097, gene#1156, sirna#1215, well_type#1274, image_idx#1333, object_idx#1392, cells_areashape_area#4106, cells_areashape_eccentricity#4165, cells_areashape_extent#4224, cells_areashape_formfactor#4283, cells_areashape_majoraxislength#4342, cells_areashape_minoraxislength#4401, cells_areashape_orientation#4460, cells_areashape_perimeter#4519, cells_children_bacteria_count#4578, cells_children_invasomes_count#4637, cells_location_center_x#4696, cells_location_center_y#4755, ... 34 more fields]\n+- Project [study#743, pathogen#802, library#861, design#920, replicate#979, plate#1038, well#1097, gene#1156, sirna#1215, well_type#1274, image_idx#1333, object_idx#1392, cells_areashape_area#4106, cells_areashape_eccentricity#4165, cells_areashape_extent#4224, cells_areashape_formfactor#4283, cells_areashape_ma

In [59]:
for x in df.columns:
    if x.startswith("cells"):
        df = df.withColumn(x, z_scores_udf("plate", "well", x).over(w))

AnalysisException: "Expression 'z_scores(plate#1038, well#1097, cells_areashape_area#4106)' not supported within a window function.;;\nProject [study#743, pathogen#802, library#861, design#920, replicate#979, plate#1038, well#1097, gene#1156, sirna#1215, well_type#1274, image_idx#1333, object_idx#1392, cells_areashape_area#7176, cells_areashape_eccentricity#4165, cells_areashape_extent#4224, cells_areashape_formfactor#4283, cells_areashape_majoraxislength#4342, cells_areashape_minoraxislength#4401, cells_areashape_orientation#4460, cells_areashape_perimeter#4519, cells_children_bacteria_count#4578, cells_children_invasomes_count#4637, cells_location_center_x#4696, cells_location_center_y#4755, ... 33 more fields]\n+- Project [study#743, pathogen#802, library#861, design#920, replicate#979, plate#1038, well#1097, gene#1156, sirna#1215, well_type#1274, image_idx#1333, object_idx#1392, cells_areashape_eccentricity#4165, cells_areashape_extent#4224, cells_areashape_formfactor#4283, cells_a

In [58]:
def median_polish(plate, wells, values):
    #print(plate[0])
    return values
z_scores_udf = func.udf(z_scores, DoubleType())

In [ ]:
def bscore(plate):
    
    grand_effect = 0
    median_row_effects = 0
    median_col_effects = 0
    row_effects = np.zeros(shape=plate.shape[0])
    col_effects = np.zeros(shape=plate.shape[1])

    for i in range(1000):
        row_medians = np.median(plate, 1) 
        row_effects += row_medians
        median_row_effects = np.median(row_effects)
        grand_effect += median_row_effects
        row_effects -= median_row_effects
        plate -= row_medians[:,np.newaxis] 

        col_medians = np.median(plate, 0) 
        col_effects += col_medians
        median_col_effects = np.median(col_effects)

        plate -= col_medians 

        grand_effect += median_col_effects

    plate = 
        
    return grand_effect, col_effects, row_effects, plate
 

In [ ]:
df.take(1)

In [15]:
outfile = file_name.replace(".tsv", "") + "zscored.tsv"
df.write.option("sep","\t").option("header","true").csv(outfile)

Py4JJavaError: An error occurred while calling o347.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:147)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:101)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apache

In [11]:
sc.stop()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1028, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/simondi/anaconda3/envs/py35/lib/python3.5/socket.py", line 575, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
/usr/local/spark/spark/python/pyspark/context.py:389: RuntimeWarning: Unable to clea